In [1]:
import pandas as pd
import requests
from PIL import Image
from transformers import AutoProcessor, LlavaNextForConditionalGeneration
import os

/Users/bytedance/Desktop/Project/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/bytedance/Desktop/Project/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [20]:

# 读取CSV文件
df = pd.read_excel('/Users/bytedance/Desktop/Project/consistency/data/data_v1.xlsx')

# 示例显示CSV文件的前几行
print("Original DataFrame:")
df = df[:1]
df

Original DataFrame:


,subject,relation,object,q1,q2,q3
0,The Simpsons,original broadcaster,Fox Broadcasting Company,What is the original broadcaster of The Simpsons?,What is the TV show in the picture?,What is the original broadcaster of the TV sho...


In [ ]:

# 初始化模型和处理器
model = LlavaNextForConditionalGeneration.from_pretrained("llava-hf/llava-v1.6-mistral-7b-hf")
processor = AutoProcessor.from_pretrained("llava-hf/llava-v1.6-mistral-7b-hf")

base_path = '/Users/bytedance/Desktop/Project/consistency/data/images'







Loading checkpoint shards: 100%|██████████| 4/4 [00:50<00:00, 12.65s/it]
Some kwargs in processor config are unused and will not have any effect: num_additional_image_tokens. 


In [ ]:
required_columns = ['subject', 'relation', 'object', 'q1', 'q2', 'q3']
for col in required_columns:
    if col not in df.columns:
        raise ValueError(f"Column '{col}' is missing in the CSV file")
    


In [22]:
# 定义一个函数来生成答案
def generate_answer(image_path, question):
    image = Image.open(image_path)
    inputs = processor(text=question, images=image, return_tensors="pt")
    generate_ids = model.generate(**inputs, max_new_tokens=30)
    result = processor.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
    return result

# 遍历每一行生成答案
for index, row in df.iterrows():
    s = row['subject']
    r = row['relation']
    o = row['object']
    folder_name = f"{str(index) + '_' + s.replace(' ', '_')}"
    image_folder_path = os.path.join(base_path, folder_name)
    
    image_path = os.path.join(image_folder_path, "0.jpg")
    print(image_path)
    if not os.path.exists(image_path):
        print(f"Image not found: {image_path}")
        continue

    # 从DataFrame中获取问题
    question1 = row['q1']
    question2 = row['q2']
    question3 = row['q3']
    print(1)
    # Step 1: 生成问题1的答案
    answer1 = generate_answer(image_path, f"[INST] <image>\n{question1} [/INST]")
    print(answer1)

    # Step 2: 生成问题2的答案
    answer2 = generate_answer(image_path, f"[INST] <image>\n{question2} [/INST]")
    print(answer2)

    # Step 3: 生成问题3的答案
    answer3 = generate_answer(image_path, f"[INST] <image>\n{question3} [/INST]")
    print(answer3)
    # 将答案添加到DataFrame中
    df.loc[index, 'question1_answer'] = answer1
    df.loc[index, 'question2_answer'] = answer2
    df.loc[index, 'question3_answer'] = answer3

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


/Users/bytedance/Desktop/Project/consistency/data/images/0_The_Simpsons/0.jpg
1


KeyboardInterrupt: 

In [ ]:
# 保存结果到新的CSV文件
df.to_csv('combined_output_with_answers.csv', index=False)

# 打印结果
print("\nCombined DataFrame with Answers:")
print(df.head())